# Fine tune ARC model using huggingface libaries

# nlpARC Solver

Written by: Jack Cole
Mindware Consulting, Inc. 
Year written: 2022
Contact: jackcole@mindware.mobi

In [1]:
# reload external modules automatically if they are changed
%load_ext autoreload
%autoreload 2

In [2]:
#!pip3 install -U git+https://github.com/arc-community/arc.git@dev

In [3]:
# import functools
# import random
# import time
# from arc.interface import Board, BoardPair
# from arc.utils.dataset import get_riddles, load_riddle_from_id, get_riddles_from_folder
# from arc.augmentations import functional
# from arc.augmentations.classes.color import RandomColor
# from arc.augmentations.classes.spatial import RandomCropInputAndOuput, RandomDoubleInputBoard, RandomRotate, RandomReflect
# from arc.augmentations.classes.noise import Noise
# from arc.augmentations.classes.helpers import same_aug_for_all_pairs_helper
# from arc.augmentations.vis_helpers import plot_task

# Setup for Colab

In [4]:
experiment_id = "ACW_2022_10_15_1_ARC_180M_DATASET_LT5"

In [5]:
colab_enabled = False
deepspeed_enabled = False
deepspeed_config_file = "ds_config_zero3.json"
import torch
print(torch.cuda.is_available())
#print(torch.cuda.device_count())
#print(torch.cuda.get_device_name(0))
#print(torch.cuda.current_device())
# cuda version
#print(torch.version.cuda)

True


In [6]:
if colab_enabled:
    !pip install wandb
!wandb login <your_api_key>

import wandb
wandb.login()
%env WANDB_LOG_MODEL=false


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Acer/.netrc
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mindware. Use `wandb login --relogin` to force relogin


env: WANDB_LOG_MODEL=false


In [7]:
if colab_enabled:
    if deepspeed_enabled:
        !pip install deepspeed
    !pip install datasets
    !pip install transformers
    if deepspeed_enabled:
        !pip install transformers[deepspeed]

# install widgets for jupyter
#!pip install ipywidgets    
#!pip install jupyterlab
#!pip install jupyterlab_widgets
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager



In [8]:
#use if on colab and you want to save data to Google Drive
if colab_enabled:
    from google.colab import drive
    drive.mount('/content/drive')

In [9]:
# DeepSpeed requires a distributed environment even when only one process is used.
# This emulates a launcher in the notebook
if deepspeed_enabled:
    import os

    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "9994"  # modify if RuntimeError: Address already in use
    os.environ["RANK"] = "0"
    os.environ["LOCAL_RANK"] = "0"
    os.environ["WORLD_SIZE"] = "1"

# Compatible Models

Here are a list of models that are comptible with this training method.  You will need to search huggingface to get the exact model name.

bart — BartForConditionalGeneration (BART model)
bigbird_pegasus — BigBirdPegasusForConditionalGeneration (BigBird-Pegasus model)
blenderbot — BlenderbotForConditionalGeneration (Blenderbot model)
blenderbot-small — BlenderbotSmallForConditionalGeneration (BlenderbotSmall model)
encoder-decoder — EncoderDecoderModel (Encoder decoder model)
fsmt — FSMTForConditionalGeneration (FairSeq Machine-Translation model)
led — LEDForConditionalGeneration (LED model)
longt5 — LongT5ForConditionalGeneration (LongT5 model)
m2m_100 — M2M100ForConditionalGeneration (M2M100 model)
marian — MarianMTModel (Marian model)
mbart — MBartForConditionalGeneration (mBART model)
mt5 — MT5ForConditionalGeneration (MT5 model)
mvp — MvpForConditionalGeneration (MVP model)
nllb — M2M100ForConditionalGeneration (NLLB model)
pegasus — PegasusForConditionalGeneration (Pegasus model)
plbart — PLBartForConditionalGeneration (PLBart model)
prophetnet — ProphetNetForConditionalGeneration (ProphetNet model)
t5 — T5ForConditionalGeneration (T5 model)
xlm-prophetnet — XLMProphetNetForConditionalGeneration (XLM-ProphetNet model)
The model is set in evaluation mode by default using model.eval() (so for instance, dropout modules are deactivated). To train the model, you should first set it back in training mode with model.train()


# Set Global Variables and Config

In [10]:
#good for testing to run through more quickly
#model_to_fine_tune = "hf-internal-testing/tiny-random-bart"

#model_to_fine_tune = "Stancld/longt5-tglobal-large-16384-pubmed-3k_steps" # 770M parameters
#model_to_fine_tune = "google/long-t5-tglobal-xl" # 3B parameters
#model_to_fine_tune = "google/long-t5-local-base" # local attention doesn't perform as well
model_to_fine_tune = "google/long-t5-tglobal-base"
tokenizer_to_use = model_to_fine_tune # "arc_longt5_tokenizer"
#model_to_fine_tune = "google/bigbird-pegasus-large-arxiv"
#model_to_fine_tune = "allenai/led-base-16384" # longformer encoder decoder - easier to train than bigbird
#model_to_fine_tune = "allenai/led-large-16384-arxiv" # 459,801,600 parameters
#model_to_fine_tune = "facebook/bart-large-cnn"
#model_to_fine_tune = "facebook/bart-large"
#model_to_fine_tune = "facebook/bart-base"
base_model = model_to_fine_tune

%env WANDB_NOTEBOOK_NAME = f"Experiment {experiment_id} {model_to_fine_tune.replace("/", "-")}"

'''
=-=-=-=-=-=-=-=-=-=-=-
iterative mode options
=-=-=-=-=-=-=-=-=-=-=-
'''
# off - no iterative fine tuning
# single model - fine tune a single model with the outputs of the previous model + prompt as inputs
# chained models - fine tune a single model with the outputs of the previous model as inputs
iterative_mode = 'off'  

accuracy_filtering_mode = False # filter out items for iterative training that are not accurate enough
# if above is false, the following 3 don't apply
accuracy_threshold_to_iterate_start = 0.4 # must be at least this accurate to be included in the next iteration
accuracy_threshold_to_iterate_current = 0.4
accuracy_threshold_step = 0.02 # must be at least this accurate to be included in the next iteration
'''
=-=-=-=-=-=-=-=-=-=-=-
end iterative mode options
=-=-=-=-=-=-=-=-=-=-=-
'''


#model_to_fine_tune = "./models/6_1_1_1" # is actually 4 epochs not 6
max_input_length = 3600
max_output_length = 500
epoch_start_up=0 # number of epochs to train for testing the code - it will run this number of epochs and then run the prediction phase on the validation data
epoch_start=0 # starting epoch number for training - use this if you want to continue training from a checkpoint
epoch_current_count=0 # tracks the current epoch number - similar to epoch_start but is incremented after each epoch
epoch_end=20 # ending epoch number for training
epoch_step=1 # step size for epochs - train for this number of epochs and then run the prediction phase on the validation data
filter_by_token_count = True # filter out items that are too long for the model
prediction_temp = 0.0 # temperature for prediction

correct_item_count_to_save_model=6 # number of correct items to save that version of the model

batch_size = 4 # batch_size of 1 works best for ARC fine tuning training

# for eval_batch_size the higher the batch size the more memory is needed; but it runs faster with a higher batch size
if colab_enabled:
    eval_batch_size = 10
else:
    eval_batch_size = 20 # this is for iterative training when the training data is run through the model to generate the training data for the next epoch

lr = 3.1e-5 # learning rate
gradient_checkpointing = True # gradient checkpointing does not work with some models - it can help to reduce memory requirements
max_checkpoint = 10 # maximum number of checkpoints to save - this can take up a lot of space if you have a large model

grid_starting_row = 4 # where the grid starts in the generated answer
output_contains_grid_size = True # if the output contains the grid size

if colab_enabled:
    experiment_folder='/content/drive/My Drive/Colab Notebooks/Experiment ACW - 180M Dataset for ARC/'
else:
    experiment_folder='./'
#tokenizer_to_use = f"{experiment_folder}{tokenizer_to_use}"
arc_data_folder = r"C:\Users\Acer\.arc\cache\dataset\sort_of_ARC_all" # folder where the ARC data files are located
#load the model from the experiment folder
#model_to_fine_tune = experiment_folder  + 'models/1_1_1_25'
#model_to_fine_tune = r"C:\Users\Acer\Desktop\ARC Challenge\Experiments\Experiment ACW - Colab DL 180M Dataset for ARC\resultsACW_2022_10_15_1_ARC_180M_DATASET_LT5\checkpoint-17500"
model_to_fine_tune = r"C:\Users\Acer\Desktop\ARC Challenge\Experiments\Experiment ACW - Colab DL 180M Dataset for ARC\models\9_3_3_56"
# use the url to get the dataset if you don't already have it
#training_data_file = 'train_plus_copy_onl_none_copy.csv' 
#training_data_file = 'train_data_len_grid_size_answers_all_output_grid_characteristics_augmente_none_copy_flip_vertically_swap.csv'  #'train_plus_copy_onl_none_copy.csv' 
#validation_data_file = 'evaluation_data_len_grid_size_answers_all_output_grid_characteristics.csv' 
training_data_file = 'train_with_letters_none_noise.csv'
validation_data_file = 'eval_data_none.csv' 
evaluation_files=[validation_data_file]
# if you want to run the training data through the model, use
#evaluation_files=[training_data_file] or evaluation_files=[training_data_file,validation_data_file] for both
import os
# check to see if training and validation files exist
if not os.path.exists(f"{experiment_folder}{training_data_file}"):
    print(f"Training file {training_data_file} does not exist")
    exit()
else:
    print("Training file exists")    
if not os.path.exists(f"{experiment_folder}{validation_data_file}"):
    print(f"Validation file {validation_data_file} does not exist")
    exit()
else:
    print("validation file exists")

# you can use this in file names for writing output
model_name_safe = base_model.replace("/", "-")

wandb.init(project="arc", name=experiment_id+"_"+model_name_safe, config={"epochs": epoch_end, "batch_size": batch_size, "train_data_file": training_data_file})

env: WANDB_NOTEBOOK_NAME=f"Experiment ACW_2022_10_15_1_ARC_180M_DATASET_LT5 google-long-t5-tglobal-base"
Training file exists
validation file exists


# Dataset details
The dataset is built, in part, from the original ARC data.  It is formatted in the way that is the most familiar to language models (i.e., like a sentence).  Here is an example prompt:

train input1 98889 88288 82228 88288 98889 output1 109 10 10 982 9888992892 8828892892 8222892892 8828892892 9888992892 9999922892 2222228892 8888888992 9999999922 2222222228. input2 23332 33533 35553 33533 23332 output2 109 10 10 235 2333225325 3353325325 3555325325 3353325325 2333225325 2222255325 5555553325 3333333225 2222222255 5555555553. test tinput1 51115 11911 19991 11911 51115 toutput1 

The choice of numbers was empirically derived through many experiments.  These experiments compared letters with commas between them, spaces between them, two character chunks, and the same options with numbers.  Numbers without spaces was slightly better than lower case letters without spaces.  With spaces and two character chunks clearly performed worse.  The row separation of a space was also derived from experiments that compared commas, new lines, and spaces.  Each input/output pair is written like a sentence and ends in a period.  This is done to help the model generate an output that has a clear ending.

Notice that the output grids have extra numbers before the grid starts.  This is the total grid length in characters, grid width, grid height, unique colors in the order they are encountered.  Ultimately, the grid length and grid size in the final answer is used to generate additional answers by parsing the grid using these predictions.  Sometimes there is a divergence, and one of these alternatively parsed grids will be correct.

Example answer:
 109 10 10 519 5111559159 1191159159 1999159159 1191159159 5111559159 5555599159 9999991159 1111111559 5555555599 9999999991. 


# Set up the GPU

In [11]:
# run nvidia-smi to see if you have a GPU
!nvidia-smi

# Setting up the device for GPU usage
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "1"

from torch import cuda
#print(torch.version.cuda)
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

Sun Oct 16 13:41:02 2022       
cuda+-----------------------------------------------------------------------------+

| NVIDIA-SMI 516.01       Driver Version: 516.01       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |

In [12]:
# import a garbage collector to free up memory
import gc

def free_memory():
    gc.collect()
    torch.cuda.empty_cache()


|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   47C    P5    47W / 350W |    998MiB / 24576MiB |     19%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
|        ID   ID                                                   Usage      |
|=============================================================================|
|    0   N/A  N/A      7256    C+G   ..

# Load the Tokenizer

In [13]:
def load_tokenizer():
  global tokenizer
  from transformers import AutoTokenizer
  print(f"loading tokenizer for arc_longt5_tokenizer")
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_to_use)

load_tokenizer()

loading tokenizer for arc_longt5_tokenizer


In [14]:
# tokenize quickly all at once
# import pandas as pd
# train_df = pd.read_csv(f"{experiment_folder}{training_data_file}")
# train_df_tokenized = tokenizer(train_df['prompt'].tolist(), padding=True, truncation=True, max_length=max_input_length, return_tensors="pt")


# Get Token Counts and Filter the data
This will use the tokenizer to tokenize the data and will filter the input prompt and correct answers by the length you specified in max_input_length and max_output_length.

TODO: Automatically get these characteristics for the model to set max_input_length and max_output_length.  Within limits though, because you don't need 16000 tokens for longt5.  3600 is sufficient for all items on input and 500 on output.

In [15]:
import pandas as pd
from datasets import Dataset

def filter_file_for_max_tokens(file_name, shuffle=True):
    
    if "http" in file_name:
      tdf = file_name
    else:
      if "/" not in file_name:
        tdf = f"{experiment_folder}{file_name}"
      else:
        tdf = file_name      

    print(f"filter_file_for_max_tokens {tdf}")
    df = pd.read_csv(tdf)

    if filter_by_token_count:
        # add the token count columns
        #tokenized_prompts = tokenizer(df['prompt'].tolist(), padding=True, truncation=True, max_length=max_input_length, return_tensors="pt")
        #df['token_count'] = df['prompt'].apply(lambda x: len(tokenized_prompts['input_ids']))
        df['token_count'] = df['prompt'].apply(lambda x: len(tokenizer.encode(x)))
        # same for correct answer
        df['token_count_correct'] = df['correct_answer'].apply(lambda x: len(tokenizer.encode(x)))

        # filter out the examples that are too long
        df = df[df['token_count'] <= max_input_length]
        df = df[df['token_count_correct'] <= max_output_length]
        # fix the index
        df = df.reset_index(drop=True)
    # randomize the dataset order
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)
    return df

prefix = ""
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["prompt"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["correct_answer"], max_length=max_output_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs    

def load_and_process_dataset_from_file(file_name, shuffle=True):
    print(f"load_and_process_dataset_from_file {experiment_folder}{file_name}")
    df = filter_file_for_max_tokens(file_name, shuffle)
    print("loading dataset from pandas dataframe")
    dataset = Dataset.from_pandas(df)
    dataset = dataset.map(preprocess_function, batched=True)
    return dataset

def load_and_process_datasets(train = training_data_file, eval = validation_data_file):
    # load dataset from the dataframes; filter out examples that are too long
    # train_dataset = Dataset.from_pandas(filter_file_for_max_tokens(train))
    # validation_dataset = Dataset.from_pandas(filter_file_for_max_tokens(eval))
    # tokenized_train = train_dataset.map(preprocess_function, batched=True)
    # tokenized_eval = validation_dataset.map(preprocess_function, batched=True)
    tokenized_train = load_and_process_dataset_from_file(train)
    tokenized_eval = load_and_process_dataset_from_file(eval)
    return tokenized_train, tokenized_eval

# Performs Inference, Data Parsing, and Item Stats

TODO: Refactor.  This is messy code. The worst are parse_answers_model1 and get_and_save_predictions.

In [16]:
import regex as re

# split the string into an array of rows and calculate the character by character accuracy between the two arrays
def calculate_character_accuracy_by_row(predicted_string, actual_string):
    # split the strings by spaces
    predicted_string_split = predicted_string.strip().split(' ')
    actual_string_split = actual_string.strip().split(' ')
    # remove empty arrays
    predicted_string_split = remove_empty_arrays(predicted_string_split)
    actual_string_split = remove_empty_arrays(actual_string_split)
    # calculate character by character accuracy
    character_accuracy = 0
    for i in range(len(predicted_string_split)):
        if i<len(actual_string_split):
            character_accuracy += calculate_character_accuracy(predicted_string_split[i], actual_string_split[i])
    if len(actual_string_split)==0:
      return 0
    else:
      return character_accuracy / len(actual_string_split)

# calculate character by character accuracy
def calculate_character_accuracy(predicted_string, actual_string):
    predicted_string=predicted_string.strip()
    actual_string=actual_string.strip()
    # calculate character by character accuracy
    character_accuracy = 0
    for i in range(len(predicted_string)):
        if i<len(actual_string):
            if predicted_string[i] == actual_string[i]:
                character_accuracy += 1
    if len(actual_string)==0:
      return 0
    else:
      return character_accuracy / len(actual_string)

# calculate row level accuracy (must by in the correct order)
def calculate_row_level_accuracy_ordered(predicted_string, actual_string):
    # split the strings by spaces
    predicted_string_split = predicted_string.strip().split(' ')
    actual_string_split = actual_string.strip().split(' ')
    # calculate word content accuracy ignoring order
    word_content_accuracy_ignoring_order = 0
    for i in range(len(predicted_string_split)):
        if i<len(actual_string_split):
            if predicted_string_split[i] == actual_string_split[i]:
                word_content_accuracy_ignoring_order += 1
    if len(actual_string_split)==0:
        return 0
    else:
        return word_content_accuracy_ignoring_order / len(actual_string_split)

# calculate row level accuracy (the rows can be in any order)
def calculate_row_level_accuracy_unordered(predicted_string, actual_string):
    predicted_string=predicted_string.strip()
    actual_string=actual_string.strip()
    if predicted_string=='':
      return 0
    # split the strings by spaces
    predicted_string_split = predicted_string.split(' ')
    actual_string_split = actual_string.split(' ')
    # remove empty arrays
    predicted_string_split = remove_empty_arrays(predicted_string_split)
    actual_string_split = remove_empty_arrays(actual_string_split)
    # calculate word content accuracy ignoring order
    word_content_accuracy_ignoring_order = 0
    for i in range(len(predicted_string_split)):
        if i<len(actual_string_split):
            # determine if the predicted word is in the actual string
            if predicted_string_split[i] in actual_string_split:
                word_content_accuracy_ignoring_order += 1
    if len(actual_string_split)==0:
        return 0
    else:
        return word_content_accuracy_ignoring_order / len(actual_string_split)

#more likely to be correct with variable sized output for rows
def trim_rows_to_smallest(grid_rows):
    #trim the strings in grid_rows to the smallest length
    smallest_length = len(grid_rows[0])
    for i in range(len(grid_rows)):
        if len(grid_rows[i]) < smallest_length:
            smallest_length = len(grid_rows[i])
    for i in range(len(grid_rows)):
        grid_rows[i] = grid_rows[i][:smallest_length]
    return grid_rows

# trim / pad to grid size
def trim_pad_to_grid_size(grid_rows, grid_row_count, grid_column_count):
    #trim the strings in grid_rows to the smallest length
    if grid_row_count>30:
        grid_row_count=30
    if grid_column_count>30:
        grid_column_count=30
    # drop rows that are beyond the row count
    grid_rows = grid_rows[:grid_row_count]
    for i in range(grid_row_count):
        if i<len(grid_rows):
            grid_rows[i] = grid_rows[i][:grid_column_count]
            # pad out to grid_column_count with 0'SCRIPT_INFO
            zeros_to_add = ''.join(['0' for x in range(grid_column_count - len(grid_rows[i]))])
            grid_rows[i] = grid_rows[i] + zeros_to_add
        else:
            # add some empty row(s) if there are fewer than grid_row_count rows
            grid_rows.append(['0' for x in range(grid_column_count)])
            # join the rows into a single string
            grid_rows[i] = ''.join(grid_rows[i])
    return grid_rows

#remove empty arrays
def remove_empty_arrays(array):
    return [x for x in array if x != '']
    
#remove leading and trailing spaces from all strings in an array
def remove_leading_and_trailing_spaces(array):
    return [x.strip() for x in array]

#split the string by anything other than a space or a number. strip leading or trailing spaces. remove empty arrays.
def split_by_non_numbers_and_not_spaces(string):
    """return a list of numbers split by anything other than numbers and spaces"""
    return remove_leading_and_trailing_spaces(remove_empty_arrays(re.split(r'[^\d\s]', string)))

# get the output from the string returned by the model
def output_string_to_answer(outputs):
           #if output_val contains "." then get the first part of the string before the "."
    if "." in outputs:
        # make sure it is a string
        output_val = str(outputs).split(".")[0]
        #output_val=outputs[0]['summary_texts'].split(".")[0]
    else:
        output_val = str(outputs)
        #output_val=outputs[0]['summary_texts']
    # trim spaces
    output_val = output_val.strip()
    return output_val

#parse the answers from the large model
def parse_answers_model1(outputs):
    output_val = output_string_to_answer(outputs)
    # check to see if it has the minimum number of words, which is grid_starting_row+2
    if len(output_val.split()) >= grid_starting_row+1:
        try:
            error = False
            output_len = int(output_val.split(" ")[0]) #subtract 1 to account for the period that is removed
        except ValueError:
            error = True
        if error == False:
            # if the grid starting column is > 1 then get the words before the grid starting column
            if grid_starting_row > 1:
                other_preds = output_val.split(" ")[:grid_starting_row]
                # remove first element (the number)
                other_preds.pop(0)
            else:
                other_preds = []
            output_val = split_by_non_numbers_and_not_spaces(output_val)[0]
            output_val = output_val.split(" ")[grid_starting_row:]
            # reassemble the list of words
            output_val = " ".join(output_val)
            try:
                output_len = int(output_len)
            except ValueError:
                output_len = len(output_val)
            try:
                # if the output is longer than the grid then trim it to the grid sizet            
                output_val_len_by_prediction=output_val[:output_len]
                output_rows_full_pred_len_limited = output_val_len_by_prediction.split(" ")
                output_rows_full_pred_len_limited=remove_empty_arrays(output_rows_full_pred_len_limited)
                output_rows = remove_empty_arrays(output_val.split(" "))
                #if the last element of output_rows is shorter than the first element, remove the last element.
                '''
                # if output_rows is a list
                if isinstance(output_rows, list):
                    if len(output_rows[-1]) < len(output_rows[0]):
                        output_rows = output_rows[:-1]
                '''
                if output_contains_grid_size and other_preds[0].isdigit() and other_preds[1].isdigit():
                    # if the output contains the grid size, then trim the output to the grid size
                    output_rows = trim_pad_to_grid_size(output_rows, int(other_preds[0]), int(other_preds[1]))
                else:
                    output_rows = trim_rows_to_smallest(output_rows)
                try:
                    error = False
                    output_len_current=len(output_rows[0])
                except IndexError:
                    output_val_len_by_prediction=output_val
                    output_len=len(output_val)
                    # add 0's to other_preds for up to grid_starting_row-1
                    other_preds = [0 for i in range(grid_starting_row)]
                    output_val_by_grid_size_pred = output_val
                if not error:
                    output_rows_full_pred_len_limited=trim_rows_to_smallest(output_rows_full_pred_len_limited)
                    #recombine the elements of output_rows into a single string.
                    output_val_len_by_prediction = " ".join(output_rows_full_pred_len_limited)     
                    output_val_by_grid_size_pred = " ".join(output_rows)
                    #fix row lengths for the first answer
                    output_rows_answer1 = output_val.split(" ")
                    output_rows_answer1=trim_rows_to_smallest(output_rows_answer1)
                    output_val = " ".join(output_rows_answer1) 
                    error=False
            except IndexError:
                error = True
                output_val_len_by_prediction=output_val
                output_len=len(output_val)
                # add 0's to other_preds for up to grid_starting_row-1
                other_preds = [0 for i in range(grid_starting_row)]
                output_val_by_grid_size_pred = output_val
        else:
            error = True
            output_val_len_by_prediction=output_val
            output_len=len(output_val)
            # add 0's to other_preds for up to grid_starting_row-1
            other_preds = [0 for i in range(grid_starting_row)]
            output_val_by_grid_size_pred = output_val
    else:
        output_val_len_by_prediction=output_val
        output_len=len(output_val)
        # add 0's to other_preds for up to grid_starting_row-1
        other_preds = [0 for i in range(grid_starting_row)]
        output_val_by_grid_size_pred = output_val
        error=True
    return output_val, output_val_len_by_prediction, output_val_by_grid_size_pred, output_len, other_preds, error

# Repeat a string n times with a comma between each repetition. It should also put a number on the end of each repetition.
def repeat_string_with_comma(string, n):
    return ", ".join([str(i+1) + "_" + string for i in range(n)])

# not using this function anymore. it is replaced by run_inference
# def do_inference(model, input_text, return_error_status=True):
#     input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
#     error = False
#     try:
#         outputs = model.generate(input_ids, temperature=prediction_temp, num_beams=4, num_beam_groups=2, top_k=50, top_p=0.95, early_stopping=True, max_length=max_output_length, repetition_penalty=2.5)
#     except IndexError:
#         outputs = None
#         error = True
#         if return_error_status:
#             return outputs, error
#         else:
#             return outputs
#     decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     # remove all characters that are not numbers, spaces, or commas
#     decoded_output = re.sub(r"[^0-9, ]", "", decoded_output) + "."
#     if return_error_status:
#         return decoded_output, error
#     else:
#         return decoded_output
    

def get_and_save_predictions(model, file):
    global last_evaluation_df # this holds the results of the last evaluation -- used with chained models
    free_memory()
    #evaluation_files=[validation_data_file]
    base_path=f"{experiment_folder}"
    sum_correct_answer=0
    sum_correct_answer_by_actual_len=0
    sum_correct_answer_len=0
    sum_correct_answer_pred_len=0
    sum_correct_answer_by_pred_grid_size=0
    sum_other_preds=[]
    sum_item_character_accuracy_ordered_unmodified=0
    sum_item_character_accuracy_ordered_unmodified_first_half=0
    sum_item_character_accuracy_ordered_unmodified_second_half=0
    sum_item_character_accuracy_ordered_by_grid_size_pred=0
    sum_item_character_accuracy_ordered_by_len_pred=0
    sum_item_character_accuracy_ordered_by_len_actual=0
    sum_row_accuracy_ordered=0
    sum_row_accuracy_unordered=0
    solved_items=[]
    for i in range(grid_starting_row-1):
        sum_other_preds.append(0)

    #data_to_evaluate=filter_file_for_max_tokens(file)
    data_to_evaluate = run_inference(model, file, False)
    free_memory()
    # remove the extension from file
    file_name_minus_extensions = file.split("/")[-1].split(".")[0]
    result_file_name = os.path.join(base_path, f"{file_name_minus_extensions}{model_name_safe}_{epoch_current_count}_epochs_{prediction_temp}_temp_{accuracy_threshold_to_iterate_current}_acc_threshold_batchsize_1_results.csv")
    #iterate through the rows in the dataframe
    for i, row in data_to_evaluate.iterrows():
        arc_fn=row["id"]
        # check to see if data_to_evaluate contains the column "augmentation"
        aug = "none"
        if "augmentation" in data_to_evaluate.columns:
            aug = row["augmentation"]
        prompt = row["prompt"]
        correct_answer = row["correct_answer"]
        #print(f"Prompt: {prompt}")
        #print(f"Correct Answer: {correct_answer}")
        outputs = row['predictions']    #, error = do_inference(model, prompt)
        #Print(f"Generated Answer: {outputs}")
        with open(result_file_name, 'a') as f:
            answer_no_non_programmatic_len_limit, answer_len_limit_by_prediction, answer_grid_size_by_prediction, output_len, other_preds, error = parse_answers_model1(outputs)
            correct_answer=correct_answer.strip().replace(".", "") #.replace(" ", "").replace(".", "")
            correct_other_preds = correct_answer.split(" ")[0:grid_starting_row]
            correct_len = int(correct_other_preds[0])
            #drop first item (length of answer)
            correct_other_preds = correct_other_preds[1:]
            #remove other preds from the correct answer
            correct_answer = correct_answer.split(" ")[grid_starting_row:len(correct_answer.split(" "))]
            correct_answer = " ".join(correct_answer)

            #don't calc any stats if there is an error
            if error==False:
                #make a boolean comparison of all items
                correct_other_preds_bool = [1 if x == y else 0 for x, y in zip(correct_other_preds, other_preds)]
                output_val_len_limited_by_correct_answer=answer_no_non_programmatic_len_limit[:len(correct_answer)]
                #convert boolean to number for csv
                correct_answer_bylen_bool=1 if correct_answer==output_val_len_limited_by_correct_answer else 0
                correct_answer_bool=1 if correct_answer==answer_no_non_programmatic_len_limit else 0
                correct_answer_bypredlen_bool=1 if correct_answer==answer_len_limit_by_prediction else 0
                correct_answer_bypredgrid_bool=1 if correct_answer==answer_grid_size_by_prediction else 0
                if correct_answer_bool or correct_answer_bylen_bool or correct_answer_bypredlen_bool or correct_answer_bypredgrid_bool:
                    solved_items.append(arc_fn)
                sum_correct_answer+=correct_answer_bool
                sum_correct_answer_by_actual_len+=correct_answer_bylen_bool
                sum_correct_answer_pred_len+=correct_answer_bypredlen_bool
                sum_correct_answer_by_pred_grid_size+=correct_answer_bypredgrid_bool
                correct_answer_len=1 if correct_len==output_len else 0
                sum_correct_answer_len+=correct_answer_len
                sum_other_preds=[x+y for x, y in zip(sum_other_preds, correct_other_preds_bool)]
                current_item_character_accuracy_ordered_unmodified=calculate_character_accuracy_by_row(answer_no_non_programmatic_len_limit, correct_answer)
                sum_item_character_accuracy_ordered_unmodified+=current_item_character_accuracy_ordered_unmodified
                sum_item_character_accuracy_ordered_unmodified_first_half+=calculate_character_accuracy_by_row(answer_no_non_programmatic_len_limit[0:int(len(answer_no_non_programmatic_len_limit)/2)], correct_answer[0:int(len(correct_answer)/2)])
                sum_item_character_accuracy_ordered_unmodified_second_half+=calculate_character_accuracy_by_row(answer_no_non_programmatic_len_limit[int(len(answer_no_non_programmatic_len_limit)/2):len(answer_no_non_programmatic_len_limit)], correct_answer[int(len(correct_answer)/2):len(correct_answer)])
                current_item_character_accuracy_ordered_by_grid_size_pred=calculate_character_accuracy_by_row(answer_grid_size_by_prediction, correct_answer)
                sum_item_character_accuracy_ordered_by_grid_size_pred+=current_item_character_accuracy_ordered_by_grid_size_pred
                current_item_character_accuracy_ordered_by_len_pred=calculate_character_accuracy_by_row(answer_len_limit_by_prediction, correct_answer)
                sum_item_character_accuracy_ordered_by_len_pred+=current_item_character_accuracy_ordered_by_len_pred
                current_item_character_accuracy_ordered_by_len_actual=calculate_character_accuracy_by_row(output_val_len_limited_by_correct_answer, correct_answer)
                sum_item_character_accuracy_ordered_by_len_actual+=current_item_character_accuracy_ordered_by_len_actual
                current_row_accuracy_ordered=calculate_row_level_accuracy_ordered(answer_no_non_programmatic_len_limit, correct_answer)
                sum_row_accuracy_ordered+=current_row_accuracy_ordered
                current_row_accuracy_unordered=calculate_row_level_accuracy_unordered(answer_no_non_programmatic_len_limit, correct_answer)
                sum_row_accuracy_unordered+=current_row_accuracy_unordered

                #if writing the first line, write the headers
                if i==0:
                    if grid_starting_row>1:
                    # remove the curly braces for the headers
                        other_preds_column_titles = repeat_string_with_comma("other_preds_string", grid_starting_row-1)
                        correct_other_preds_string_column_titles = repeat_string_with_comma("correct_other_preds_string", grid_starting_row-1)
                        correct_other_preds_bool_column_titles = repeat_string_with_comma("correct_other_preds_bool", grid_starting_row-1)
                        # some of the headers don't correspond exactly to the variable names for interpretability
                        header_string='{arc_fn},{augmentation},{prompt},{correct_answer},{gen_answer_no_non_programmatic_len_limit},{gen_answer_len_limit_by_prediction},{gen_answer_grid_size_by_prediction},{gen_answer_len_limited_by_correct_answer_len},{correct_answer_len},{predicted_output_len},{pred_len_equals_actual_len_bool},'+other_preds_column_titles+','+correct_other_preds_string_column_titles+','+correct_other_preds_bool_column_titles+',{pred_answer_unmodified_equals_correct_answer_bool},{pred_answer_parsed_by_correct_answer_len_equals_correct_answer_bool},{pred_answer_parsed_by_pred_grid_size_equals_correct_answer_bool},{pred_answer_parsed_by_pred_len_equals_correct_answer_bool},{current_item_character_accuracy_ordered_unmodified},{current_item_character_accuracy_ordered_by_grid_size_pred},{current_item_character_accuracy_ordered_by_len_pred},{current_item_character_accuracy_ordered_by_len_actual},{current_row_accuracy_ordered},{current_row_accuracy_unordered},{raw_unparsed_prediction}\n'
                    else:
                        header_string='{arc_fn},{augmentation},{prompt},{correct_answer},{gen_answer_no_non_programmatic_len_limit},{gen_answer_len_limit_by_prediction},{gen_answer_len_limited_by_correct_answer_len},{correct_answer_len},{predicted_output_len},{pred_len_equals_actual_len_bool},{pred_answer_unmodified_equals_correct_answer_bool},{pred_answer_parsed_by_correct_answer_len_equals_correct_answer_bool},{pred_answer_parsed_by_pred_len_equals_correct_answer_bool},{current_item_character_accuracy_ordered_unmodified},{current_item_character_accuracy_ordered_by_len_pred},{current_item_character_accuracy_ordered_by_len_actual},{current_row_accuracy_ordered},{current_row_accuracy_unordered},{raw_unparsed_prediction}\n'
                    # write the header_string
                    header_string = header_string.replace("{", "").replace("}", "")
                    f.write(header_string)
                other_preds_string = ",".join(str(x) for x in other_preds)
                correct_other_preds_string = ",".join(str(x) for x in correct_other_preds)
                correct_other_preds_bool_string = ",".join(str(x) for x in correct_other_preds_bool)
                if grid_starting_row>1:
                    #{other_preds_string},{correct_other_preds_string},{sum_other_preds_string}
                    string_to_write=f"{arc_fn},{aug},{prompt},{correct_answer},{answer_no_non_programmatic_len_limit},{answer_len_limit_by_prediction},{answer_grid_size_by_prediction},{output_val_len_limited_by_correct_answer},{correct_len},{output_len},{correct_answer_len},{other_preds_string},{correct_other_preds_string},{correct_other_preds_bool_string},{correct_answer_bool},{correct_answer_bylen_bool},{correct_answer_bypredgrid_bool},{correct_answer_bypredlen_bool},{current_item_character_accuracy_ordered_unmodified},{current_item_character_accuracy_ordered_by_grid_size_pred},{current_item_character_accuracy_ordered_by_len_pred},{current_item_character_accuracy_ordered_by_len_actual},{current_row_accuracy_ordered},{current_row_accuracy_unordered},{outputs}\n"
                else:
                    string_to_write=f"{arc_fn},{aug},{prompt},{correct_answer},{answer_no_non_programmatic_len_limit},{answer_len_limit_by_prediction},{output_val_len_limited_by_correct_answer},{correct_len},{output_len},{correct_answer_len},{correct_answer_bool},{correct_answer_bylen_bool},{correct_answer_bypredlen_bool},{current_item_character_accuracy_ordered_unmodified},{current_item_character_accuracy_ordered_by_len_pred},{current_item_character_accuracy_ordered_by_len_actual},{current_row_accuracy_ordered},{current_row_accuracy_unordered},{outputs}\n"
                string_to_write=string_to_write.encode('ascii', 'ignore').decode('ascii') #occasionally crashed in the past due to unicode characters
                f.write(string_to_write)
            else: # unable to correctly parse the answer; just write the outputs unmodified
                f.write(f"{arc_fn},{aug},{prompt},{correct_answer},{output_string_to_answer(outputs)}\n")
                #write the sums
        last_evaluation_df = pd.read_csv(result_file_name)
    with open(result_file_name, 'a') as f:
        #replace the unicode characters with ascii in string_to_write
        sum_other_preds_string = ",".join(str(x) for x in sum_other_preds)
        sum_other_preds_column_titles = repeat_string_with_comma("sum_other_preds", grid_starting_row-1)
        string_to_write="\n"+sum_other_preds_column_titles+",{sum_correct_answer_len},{avg_item_character_accuracy_ordered_unmodified},{avg_item_character_accuracy_ordered_by_grid_size_pred},{avg_item_character_accuracy_ordered_unmodified_first_half},{avg_item_character_accuracy_ordered_unmodified_second_half},{avg_item_character_accuracy_ordered_by_len_pred},{avg_item_character_accuracy_ordered_by_len_actual},{avg_row_accuracy_ordered},{avg_row_accuracy_unordered},{sum_correct_answer},{sum_correct_answer_by_actual_len},{sum_correct_answer_pred_len},{sum_correct_answer_by_pred_grid_size}\n".replace("{", "").replace("}", "")
        f.write(string_to_write)
        # divide the character accuracy and row accuracy by the number of items
        # get number of items in data_to_evaluate
        num_items_in_data_to_evaluate = len(data_to_evaluate)
        sum_item_character_accuracy_ordered_unmodified/=num_items_in_data_to_evaluate
        sum_item_character_accuracy_ordered_by_grid_size_pred/=num_items_in_data_to_evaluate
        sum_item_character_accuracy_ordered_by_len_pred/=num_items_in_data_to_evaluate
        sum_item_character_accuracy_ordered_by_len_actual/=num_items_in_data_to_evaluate
        sum_item_character_accuracy_ordered_unmodified_first_half/=num_items_in_data_to_evaluate
        sum_item_character_accuracy_ordered_unmodified_second_half/=num_items_in_data_to_evaluate
        sum_row_accuracy_ordered/=num_items_in_data_to_evaluate
        sum_row_accuracy_unordered/=num_items_in_data_to_evaluate
        wandb.log({"epoch_current_count" : epoch_current_count, "sum_correct_answer": sum_correct_answer, "sum_correct_answer_by_actual_len": sum_correct_answer_by_actual_len, "sum_correct_answer_pred_len": sum_correct_answer_pred_len, "sum_correct_answer_by_pred_grid_size": sum_correct_answer_by_pred_grid_size, "sum_item_character_accuracy_ordered_unmodified": sum_item_character_accuracy_ordered_unmodified, "sum_item_character_accuracy_ordered_by_grid_size_pred": sum_item_character_accuracy_ordered_by_grid_size_pred, "sum_item_character_accuracy_ordered_by_len_pred": sum_item_character_accuracy_ordered_by_len_pred, "sum_item_character_accuracy_ordered_by_len_actual": sum_item_character_accuracy_ordered_by_len_actual, "sum_row_accuracy_ordered": sum_row_accuracy_ordered, "sum_row_accuracy_unordered": sum_row_accuracy_unordered, "solved_items" : len(solved_items)})
        string_to_write=f"{sum_other_preds_string},{sum_correct_answer_len},{sum_item_character_accuracy_ordered_unmodified},{sum_item_character_accuracy_ordered_by_grid_size_pred},{sum_item_character_accuracy_ordered_unmodified_first_half},{sum_item_character_accuracy_ordered_unmodified_second_half},{sum_item_character_accuracy_ordered_by_len_pred},{sum_item_character_accuracy_ordered_by_len_actual},{sum_row_accuracy_ordered},{sum_row_accuracy_unordered},{sum_correct_answer},{sum_correct_answer_by_actual_len},{sum_correct_answer_pred_len},{sum_correct_answer_by_pred_grid_size}\n"
        f.write(string_to_write)
    return sum_correct_answer, sum_correct_answer_by_actual_len, sum_correct_answer_pred_len, sum_correct_answer_by_pred_grid_size, sum_correct_answer_len, sum_other_preds_string, data_to_evaluate, sum_item_character_accuracy_ordered_unmodified, sum_item_character_accuracy_ordered_by_grid_size_pred, sum_item_character_accuracy_ordered_by_len_pred, sum_item_character_accuracy_ordered_by_len_actual, sum_row_accuracy_ordered, sum_row_accuracy_unordered, solved_items, error

In [17]:
#get_and_save_predictions(model, evaluation_files[1])

# Train the Model and Related Functions

The training will train for epoch_steps and then will run the validation data through the model to generate the outputs and custom stats.  This is saved to a file after every epoch_steps.  The model is also saved after every epoch_steps.  

Functions to Train and Save Overall Custom Stats

In [18]:
#from transformers.utils.dummy_tf_objects import TFDebertaV2ForSequenceClassification
#from distutils.command.config import config

# need to combine the answers from the last evaluation run with the prompts to train the next model
def create_new_evaluation_dataset_for_chained_models(parse_prior_answers=True):
    global last_evaluation_df
    # combine the columns prompt and raw_unparsed_prediction
    # if the dataframe does not contain the column prompt_original, then it is the first model in the chain, so save it to prompt_original
    if 'prompt_original' not in last_evaluation_df.columns:
        last_evaluation_df['prompt_original'] = last_evaluation_df['prompt']
    if parse_prior_answers:
        last_evaluation_df['prompt'] = last_evaluation_df['prompt_original'] + last_evaluation_df['raw_unparsed_prediction']
    else:
        last_evaluation_df['prompt'] = last_evaluation_df['prompt'] + last_evaluation_df['raw_unparsed_prediction']
    #last_evaluation_df['prompt'] = last_evaluation_df['prompt'].apply(lambda x: x.replace("  ", " ").replace("  ", " "))
    # drop rows where prompt is blank
    #last_evaluation_df = last_evaluation_df[last_evaluation_df['prompt'].str.strip() != '']
    # write to file
    new_validation_data_file = f"evaluation_dataset_for_chained_models_{epoch_current_count-epoch_step}.csv"
    last_evaluation_df.to_csv(f"{experiment_folder}{new_validation_data_file}", index=False)
    return new_validation_data_file

def create_new_iterative_training_dataset():
    print("create_new_iterative_training_dataset")
    if (epoch_current_count<epoch_step+1 or iterative_mode=='off') or (iterative_mode=='chained models' and epoch_current_count==0):
        return load_and_process_datasets()
    
    # run inference on the training data; run do_inference on the training data "prompt"
    if "http" not in training_data_file:
      if "/" not in training_data_file:
        tdf = f"{experiment_folder}{training_data_file}"
      else:
        tdf = training_data_file
    else:
      tdf = training_data_file
    print("create_new_iterative_training_dataset: tdf", tdf)
    training_df = filter_file_for_max_tokens(tdf, False)
    # copy the training data to a new dataframe
    #new_training_df = training_df.copy()
    new_training_df = run_inference(model, tdf, False)
 
    # combine the prompt and the results
    new_training_df['prompt'] = new_training_df['prompt'] + new_training_df['results']
    new_training_df['prompt'] = new_training_df['prompt'].apply(lambda x: x.replace("  ", " ").replace("  ", " "))
    
    unflitered_new_df = new_training_df.copy()
    # filter out the items that have accuracy of > accuracy_threshold_to_iterate_current
    if accuracy_filtering_mode:
      new_training_df = new_training_df[new_training_df["accuracy"]>=accuracy_threshold_to_iterate_current]

    new_training_file = f"train_{epoch_current_count-epoch_step}_epoch_start.csv" 
    unfiltered_new_training_file = f"train_{epoch_current_count-epoch_step}_epoch_start_unfiltered.csv"
    # add the items from the new_training_df to the training_df
    if iterative_mode!='chained models': # chained models don't include the original training data
        training_df = pd.concat([training_df, new_training_df])
        training_df.to_csv(f"{experiment_folder}{new_training_file}", index=False)
    else:
        new_training_df.to_csv(f"{experiment_folder}{new_training_file}", index=False)

    unfiltered_new_training_df = pd.concat([training_df, unflitered_new_df])
    unfiltered_new_training_df.to_csv(f"{experiment_folder}{unfiltered_new_training_file}", index=False)

    # need to save the new validation file as well
    # also need option to continue adding to the training file instead of removing the old one
    print("end create_new_iterative_training_dataset")
    #iterative_mode = 'off'  # 'single model' or 'chained models'
    if not accuracy_filtering_mode:
        new_training_file = unfiltered_new_training_file
    if iterative_mode=='chained models':
        validation_data_file = create_new_evaluation_dataset_for_chained_models(True)
    return load_and_process_datasets(new_training_file, validation_data_file)

def save_metrics(trainer, train_result, tokenized_train, tokenized_eval):
   # compute train results
    metrics = train_result.metrics
    max_train_samples = len(tokenized_train)
    metrics["train_samples"] = min(max_train_samples, len(tokenized_train))

    # save train results
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)

    # compute evaluation results
    metrics = trainer.evaluate()
    max_val_samples = len(tokenized_eval)
    metrics["eval_samples"] = min(max_val_samples, len(tokenized_eval))

    # save evaluation results
    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)

def train_for_epochs_and_calc_totals(epochs_to_train, epoch_current_count, stats_obj):
    print(f"training for {epochs_to_train} epochs with batch size {batch_size}")

    free_memory()

    if iterative_mode=='chained models':
        load_tokenizer()
        load_model() # load a fresh model each time
    
    global accuracy_threshold_to_iterate_current, accuracy_threshold_step
    accuracy_threshold_to_iterate_current += accuracy_threshold_step

    epoch_current_count+=epochs_to_train
    tokenized_train, tokenized_eval = create_new_iterative_training_dataset()
    # https://huggingface.co/docs/transformers/v4.21.2/en/main_classes/trainer#transformers.TrainingArguments
    the_output_dir = f"{experiment_folder if not colab_enabled else './'}results{experiment_id}"
    if deepspeed_enabled:
        training_args = Seq2SeqTrainingArguments(
            report_to = 'wandb',                     # enable logging to W&B
            output_dir=the_output_dir,              # output directory
            evaluation_strategy="epoch",
            learning_rate=lr,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            weight_decay=0.01,
            save_total_limit=max_checkpoint, # save manually
            num_train_epochs=epochs_to_train,
            fp16=False,
            seed=42,
            deepspeed=f"{experiment_folder}{deepspeed_config_file}",
        )
    else:
        training_args = Seq2SeqTrainingArguments(
            report_to = 'wandb',                     # enable logging to W&B
            output_dir=the_output_dir,              # output directory
            evaluation_strategy="epoch",
            learning_rate=lr,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            weight_decay=0.01,
            save_total_limit=max_checkpoint, # save manually
            num_train_epochs=epochs_to_train,
            fp16=False,
            seed=42,
        )
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    
    #trainer.evaluate()
    train_result = trainer.train()
    save_metrics(trainer, train_result, tokenized_train, tokenized_eval)
    # iterate through the evaluation files
    for file in evaluation_files:
        current_correct_count, current_correct_by_len_count, correct_answer_by_predlen_count, current_correct_by_grid_size_count, correct_pred_len, correct_other_pred, evaluation_df_and_results, character_accuracy, sum_item_character_accuracy_ordered_by_grid_size_pred, sum_item_character_accuracy_ordered_by_len_pred, sum_item_character_accuracy_ordered_by_len_actual, sum_row_accuracy_ordered, sum_row_accuracy_unordered, solved_items, error = get_and_save_predictions(model, file)
        if error==False:
            stats_obj[file].loc[len(stats_obj[file])]=[epoch_current_count,  current_correct_count, current_correct_by_len_count, correct_answer_by_predlen_count, current_correct_by_grid_size_count, correct_pred_len, correct_other_pred, character_accuracy, sum_item_character_accuracy_ordered_by_grid_size_pred, sum_item_character_accuracy_ordered_by_len_pred, sum_item_character_accuracy_ordered_by_len_actual, sum_row_accuracy_ordered, sum_row_accuracy_unordered, solved_items]
            # correct_counts.append(current_correct_count)
            # correct_counts_len.append(current_correct_by_len_count)
            epoch_current.append(epoch_current_count-epochs_to_train)
            # correct_counts_pred_len.append(correct_pred_len)
            # correct_counts_bypred_len.append(correct_answer_by_predlen_count)
            # correct_counts_other_preds.append(correct_other_pred)
            # correct_counts_by_grid_size.append(current_correct_by_grid_size_count)
            # character_level_accuracy.append(character_accuracy)
            # character_level_accuracy_by_grid_size_pred.append(sum_item_character_accuracy_ordered_by_grid_size_pred)
            # character_level_accuracy_by_len_pred.append(sum_item_character_accuracy_ordered_by_len_pred)
            # character_level_accuracy_by_len_actual.append(sum_item_character_accuracy_ordered_by_len_actual)
            # row_level_accuracy_ordered.append(sum_row_accuracy_ordered)
            # row_level_accuracy_unordered.append(sum_row_accuracy_unordered)
            # correct_items.append(solved_items)
            # if any of the most recent correct counts are > 0 then save to a new folder
            index = len(stats_obj[file])-1
            if iterative_mode=="chained models" or stats_obj[file]['correct_counts'][index]>=correct_item_count_to_save_model or stats_obj[file]['correct_counts_bypred_len'][index]>=correct_item_count_to_save_model or stats_obj[file]['correct_counts_by_grid_size'][index]>=correct_item_count_to_save_model:
                if iterative_mode=="chained models":
                    model.save_pretrained(f"{experiment_folder}/models/{epoch_current_count-epochs_to_train}")
                    tokenizer.save_pretrained(f"{experiment_folder}/models/{epoch_current_count-epochs_to_train}")
                else:    
                    model.save_pretrained(f"{experiment_folder}/models/{epoch_current_count-epochs_to_train}_{stats_obj[file]['correct_counts'][index]}_{stats_obj[file]['correct_counts_bypred_len'][index]}_{stats_obj[file]['correct_counts_by_grid_size'][index]}")
                    tokenizer.save_pretrained(f"{experiment_folder}/models/{epoch_current_count-epochs_to_train}_{stats_obj[file]['correct_counts'][index]}_{stats_obj[file]['correct_counts_bypred_len'][index]}_{stats_obj[file]['correct_counts_by_grid_size'][index]}")
            else:
                model.save_pretrained(f"{experiment_folder}")
                tokenizer.save_pretrained(f"{experiment_folder}")
            save_data_and_stats(stats_obj, file)

def save_data_and_stats(stats_obj, file):
    # results file
    if len(stats_obj[file])>0:
        # remove extension from file name
        file_name_minus_extension = file.split('.')[0]
        results_file = f"{file_name_minus_extension}{model_name_safe}_final_results.csv"
        # if file exists then set a variable
        if os.path.isfile(os.path.join(experiment_folder, results_file)):
            file_exists = True
        else:
            file_exists = False
        #write the correct_counts and correct_counts_len to a file final_results.csv
        with open(os.path.join(experiment_folder, results_file), 'a') as f:
            #write the column labels
            if not file_exists:
                f.write('model_name,epoch_start,epoch_end,epoch_step,batch_size,lr,epoch_step')
                f.write(f"{model_to_fine_tune},{epoch_start},{epoch_end},{epoch_step},{batch_size},{lr},{epoch_step}\n")
                # label the columns
                correct_other_preds_column_titles = repeat_string_with_comma("other_preds_string", grid_starting_row-1)
                f.write(f"epoch,correct_count,correct_count_by_len,correct_by_pred_len,correct_pred_len,correct_answer_by_grid_size,"+correct_other_preds_column_titles+",cell_level_accuracy,cell_level_accuracy_by_grid_size_pred,cell_level_accuracy_by_pred_len,cell_level_accuracy_by_correct_ans_len,row_level_accuracy_ordered,row_level_accuracy_unordered,solved_items\n")
            i = len(stats_obj[file])-1
            try:
                # the last item in correct_counts_pred_len
                curr_pred_len_correct=stats_obj[file]['correct_counts_pred_len'][i]
            except:
                curr_pred_len_correct=0
            
            epochval=epoch_current[i]
            f.write(f"{epochval},{stats_obj[file]['correct_counts'][i]},{stats_obj[file]['correct_counts_len'][i]},{stats_obj[file]['correct_counts_bypred_len'][i]},{stats_obj[file]['correct_counts_pred_len'][i]},{stats_obj[file]['correct_counts_by_grid_size'][i]},{stats_obj[file]['correct_counts_other_preds'][i]},{stats_obj[file]['character_level_accuracy'][i]},{stats_obj[file]['character_level_accuracy_by_grid_size_pred'][i]},{stats_obj[file]['character_level_accuracy_by_len_pred'][i]},{stats_obj[file]['character_level_accuracy_by_len_actual'][i]},{stats_obj[file]['row_level_accuracy_ordered'][i]},{stats_obj[file]['row_level_accuracy_unordered'][i]},{','.join(stats_obj[file]['correct_items'][i])}")
            f.write("\n")

In [19]:
# save the stats object if something goes wrong

# epochs_to_train = 2
# # iterate through the evaluation files
# for file in evaluation_files:
#     current_correct_count, current_correct_by_len_count, correct_answer_by_predlen_count, current_correct_by_grid_size_count, correct_pred_len, correct_other_pred, evaluation_df_and_results, character_accuracy, sum_item_character_accuracy_ordered_by_grid_size_pred, sum_item_character_accuracy_ordered_by_len_pred, sum_item_character_accuracy_ordered_by_len_actual, sum_row_accuracy_ordered, sum_row_accuracy_unordered, solved_items, error = get_and_save_predictions(model, file)
#     if error==False:
#         stats_obj[file].loc[len(stats_obj[file])]=[epoch_current_count,  current_correct_count, current_correct_by_len_count, correct_answer_by_predlen_count, current_correct_by_grid_size_count, correct_pred_len, correct_other_pred, character_accuracy, sum_item_character_accuracy_ordered_by_grid_size_pred, sum_item_character_accuracy_ordered_by_len_pred, sum_item_character_accuracy_ordered_by_len_actual, sum_row_accuracy_ordered, sum_row_accuracy_unordered, solved_items]
#         # correct_counts.append(current_correct_count)
#         # correct_counts_len.append(current_correct_by_len_count)
#         epoch_current.append(epoch_current_count-epochs_to_train)
#         # correct_counts_pred_len.append(correct_pred_len)
#         # correct_counts_bypred_len.append(correct_answer_by_predlen_count)
#         # correct_counts_other_preds.append(correct_other_pred)
#         # correct_counts_by_grid_size.append(current_correct_by_grid_size_count)
#         # character_level_accuracy.append(character_accuracy)
#         # character_level_accuracy_by_grid_size_pred.append(sum_item_character_accuracy_ordered_by_grid_size_pred)
#         # character_level_accuracy_by_len_pred.append(sum_item_character_accuracy_ordered_by_len_pred)
#         # character_level_accuracy_by_len_actual.append(sum_item_character_accuracy_ordered_by_len_actual)
#         # row_level_accuracy_ordered.append(sum_row_accuracy_ordered)
#         # row_level_accuracy_unordered.append(sum_row_accuracy_unordered)
#         # correct_items.append(solved_items)
#         # if any of the most recent correct counts are > 0 then save to a new folder
#         index = len(stats_obj[file])-1
#         if iterative_mode=="chained models" or stats_obj[file]['correct_counts'][index]>=correct_item_count_to_save_model or stats_obj[file]['correct_counts_bypred_len'][index]>=correct_item_count_to_save_model or stats_obj[file]['correct_counts_by_grid_size'][index]>=correct_item_count_to_save_model:
#             if iterative_mode=="chained models":
#                 model.save_pretrained(f"{experiment_folder}/models/{epoch_current_count-epochs_to_train}")
#                 tokenizer.save_pretrained(f"{experiment_folder}/models/{epoch_current_count-epochs_to_train}")
#             else:    
#                 model.save_pretrained(f"{experiment_folder}/models/{epoch_current_count-epochs_to_train}_{stats_obj[file]['correct_counts'][index]}_{stats_obj[file]['correct_counts_bypred_len'][index]}_{stats_obj[file]['correct_counts_by_grid_size'][index]}")
#                 tokenizer.save_pretrained(f"{experiment_folder}/models/{epoch_current_count-epochs_to_train}_{stats_obj[file]['correct_counts'][index]}_{stats_obj[file]['correct_counts_bypred_len'][index]}_{stats_obj[file]['correct_counts_by_grid_size'][index]}")
#         else:
#             model.save_pretrained(f"{experiment_folder}")
#             tokenizer.save_pretrained(f"{experiment_folder}")
#         save_data_and_stats(stats_obj, file)

Load Model and run_inference (and supporting functions)

In [20]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq

def load_model():
    global model, data_collator
    model = AutoModelForSeq2SeqLM.from_pretrained(model_to_fine_tune).to(device)
    print("model parameters: "+str(sum(p.numel() for p in model.parameters() if p.requires_grad)))
    # when it works, this causes the model to use a lot less memory
    if gradient_checkpointing:
        model.gradient_checkpointing_enable()
        model.encoder.gradient_checkpointing = True
        model.decoder.gradient_checkpointing = True

    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

class SimpleDataset: # used to wrap the data for the evaluation phase
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

def parse_out_grid(list_of_answers):
    grid_list = []
    for correct_answer in list_of_answers:
        correct_answer=correct_answer.strip().replace(".", "") #.replace(" ", "").replace(".", "")
        correct_other_preds = correct_answer.split(" ")[0:grid_starting_row]
        correct_len = int(correct_other_preds[0])
        #drop first item (length of answer)
        correct_other_preds = correct_other_preds[1:]
        #remove other preds from the correct answer
        correct_answer = correct_answer.split(" ")[grid_starting_row:len(correct_answer.split(" "))]
        correct_answer = " ".join(correct_answer)
        grid_list.append(correct_answer.strip())
    return grid_list

def calc_accuracy(predictions, dataset_file, shuffle=True):
    # use calc character accuracy to calculate the accuracy of the predictions
    # parse the answer first
    # iterate through the predictions
    df = filter_file_for_max_tokens(dataset_file, shuffle)
    df['predictions'] = predictions
    results=[]
    for prediction in predictions:
        answer_no_non_programmatic_len_limit, answer_len_limit_by_prediction, answer_grid_size_by_prediction, output_len, other_preds, error = parse_answers_model1(prediction)
        results.append(answer_grid_size_by_prediction)
    df['results'] = results
    df['answer_grid'] = parse_out_grid(df['correct_answer'])
    df['accuracy'] = df.apply(lambda row: calculate_character_accuracy(row['answer_grid'], row['results']), axis=1)
    return df

def run_inference(model, dataset_file, shuffle=True): # run a whole batch of prompts through the model
    print("===> Running inference...")
    #tokenized_texts = tokenizer(prompt_strings, return_tensors="pt")
    #test_dataset = SimpleDataset(tokenized_texts)
    test_dataset = load_and_process_dataset_from_file(dataset_file, shuffle)
    if deepspeed_enabled:
        training_args = Seq2SeqTrainingArguments(
            output_dir="./results",
            evaluation_strategy="epoch",
            per_device_train_batch_size=eval_batch_size,
            per_device_eval_batch_size=eval_batch_size,
            predict_with_generate=True,
            fp16=False,
            seed=42,
            deepspeed=f"{experiment_folder}{deepspeed_config_file}",
        )
    else:
        training_args = Seq2SeqTrainingArguments(
            output_dir="./results",
            evaluation_strategy="epoch",
            per_device_train_batch_size=eval_batch_size,
            per_device_eval_batch_size=eval_batch_size,
            predict_with_generate=True,
            fp16=False,
            seed=42,
        )
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    result = {}
    result = trainer.predict(test_dataset, temperature=prediction_temp, max_length=max_output_length) #, max_length=512
    tokenizer.batch_decode(result.predictions, skip_special_tokens=True)
    predictions = tokenizer.batch_decode(result.predictions, skip_special_tokens=True)
    return calc_accuracy(predictions, dataset_file, shuffle)  # list of strings


Train the Model

In [21]:
stats_obj = {}
# make an array for each of the evaluation_files
for evaluation_file in evaluation_files:
    stats_obj[evaluation_file] = pd.DataFrame(columns=['epoch', 'correct_counts', 'correct_counts_len', 'correct_counts_bypred_len', 'correct_counts_pred_len', 'correct_counts_by_grid_size', 'correct_counts_other_preds', 'character_level_accuracy', 'character_level_accuracy_by_grid_size_pred', 'character_level_accuracy_by_len_pred', 'character_level_accuracy_by_len_actual', 'row_level_accuracy_ordered', 'row_level_accuracy_unordered', 'correct_items'])

# correct_counts=[]
# correct_counts_len=[]
# correct_counts_bypred_len=[]
# correct_counts_pred_len=[]
# correct_counts_other_preds=[]
# correct_counts_by_grid_size=[]
# character_level_accuracy=[]
# character_level_accuracy_by_grid_size_pred=[]
# character_level_accuracy_by_len_pred=[]
# character_level_accuracy_by_len_actual=[]
# row_level_accuracy_ordered=[]
# row_level_accuracy_unordered=[]
# correct_items=[]
accuracy_threshold_to_iterate_current = accuracy_threshold_to_iterate_start
epoch_current=[]

if iterative_mode!='chained models':
    print("===> Loading model...")
    load_model()

if epoch_start_up>0:
    epoch_current_count=epoch_start_up
    train_for_epochs_and_calc_totals(epoch_start_up, epoch_current_count, stats_obj)

for epochs in range(epoch_start+1, epoch_end, epoch_step):
    epoch_current_count+=epoch_step
    train_for_epochs_and_calc_totals(epoch_step, epoch_current_count, stats_obj)


#learn.validate(1, dl=None, cbs=fit_cbs)

===> Loading model...
model parameters: 247587456
training for 1 epochs with batch size 8
create_new_iterative_training_dataset
load_and_process_dataset_from_file ./train_with_letters_none_noise.csv
filter_file_for_max_tokens ./train_with_letters_none_noise.csv
loading dataset from pandas dataframe


  0%|          | 0/75 [00:00<?, ?ba/s]

load_and_process_dataset_from_file ./eval_data_none.csv
filter_file_for_max_tokens ./eval_data_none.csv
loading dataset from pandas dataframe


  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the training set don't have a corresponding argument in `LongT5ForConditionalGeneration.forward` and have been ignored: correct_answer, token_count_correct, id, path_data, augmentation, number_to_replace_list, replace_with_list, prompt, token_count. If correct_answer, token_count_correct, id, path_data, augmentation, number_to_replace_list, replace_with_list, prompt, token_count are not expected by `LongT5ForConditionalGeneration.forward`,  you can safely ignore this message.
c:\ProgramData\Anaconda3\envs\ml\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 74137
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulati

...

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The following columns in the training set don't have a corresponding argument in `LongT5ForConditionalGeneration.forward` and have been ignored: correct_answer, token_count_correct, id, path_data, augmentation, number_to_replace_list, replace_with_list, prompt, token_count. If correct_answer, token_count_correct, id, path_data, augmentation, number_to_replace_list, replace_with_list, prompt, token_count are not expected by `LongT5ForConditionalGeneration.forward`,  you can safely ignore this message.
c:\ProgramData\Anaconda3\envs\ml\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 74137
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parall

  0%|          | 0/9268 [00:00<?, ?it/s]

c:\ProgramData\Anaconda3\envs\ml\lib\site-packages\transformers\modeling_utils.py:667: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


KeyboardInterrupt: 

# Run all above cells for fine tuning.  The cells below can be used to resume training if it is stopped

# Tools Below

In [ ]:
# this will print results
# other results files are saved in the experiment folder in csv format for each epoch_step with a file name of {model_name_safe}_{epoch_current_count}_epochs_batchsize_1_results.csv
# if you train to the end, the final results will be saved in a file called {model_name_safe}_final_results.csv

# Print results for each evaluation file
for file in evaluation_files:
    print(f"===> Results for {file}")
    print(stats_obj[file])
    print(f"Correct counts: {stats_obj[file]['correct_counts']}")
    print(f"Correct counts by len: {stats_obj[file]['correct_counts_len']}")
    print(f"Correct counts by pred len: {stats_obj[file]['correct_counts_bypred_len']}")
    print(f"Correct counts by grid size: {stats_obj[file]['correct_counts_by_grid_size']}")
    print(f"Correct prediction of length: {stats_obj[file]['correct_counts_pred_len']}")
    print(f"Correct other predictions: {stats_obj[file]['correct_counts_other_preds']}")
    print(f"Correct items: {stats_obj[file]['correct_items']}")
# summary stats

    #print the headers
    print(f"model_name,epoch_start,epoch_end,epoch_step,batch_size,lr,epoch_step")
    print(f"{model_to_fine_tune},{epoch_start},{epoch_end},{epoch_step},{batch_size},{lr},{epoch_step}")

    # iterate through the correct_counts and print the results
    correct_other_preds_column_titles = repeat_string_with_comma("other_preds_string", grid_starting_row-1)
    print(f"epoch,correct_count,correct_count_by_len,correct_by_pred_len,correct_pred_len,correct_answer_by_grid_size,"+correct_other_preds_column_titles+",cell_level_accuracy,cell_level_accuracy_by_grid_size_pred,cell_level_accuracy_by_pred_len,cell_level_accuracy_by_correct_ans_len,row_level_accuracy_ordered,row_level_accuracy_unordered,solved_items\n")
    for i in range(len(stats_obj[file]['correct_counts'])):
        epochval=epoch_current[i]
        try:
            curr_pred_len_correct=stats_obj[file]['correct_counts_pred_len'][i]
        except:
            curr_pred_len_correct=0
        print(f"{epochval},{stats_obj[file]['correct_counts'][i]},{stats_obj[file]['correct_counts_len'][i]},{stats_obj[file]['correct_counts_bypred_len'][i]},{stats_obj[file]['correct_counts_pred_len'][i]},{stats_obj[file]['correct_counts_by_grid_size'][i]},{stats_obj[file]['correct_counts_other_preds'][i]},{stats_obj[file]['character_level_accuracy'][i]},{stats_obj[file]['character_level_accuracy_by_grid_size_pred'][i]},{stats_obj[file]['character_level_accuracy_by_len_pred'][i]},{stats_obj[file]['character_level_accuracy_by_len_actual'][i]},{stats_obj[file]['row_level_accuracy_ordered'][i]},{stats_obj[file]['row_level_accuracy_unordered'][i]},{','.join(stats_obj[file]['correct_items'][i])}\n")
    print("\n")

# Visualize Solved Items

In [ ]:
# #correct_items = [["3b4c2228_0"],["3b4c2228_0", "358ba94e_0", "1a2e2828_0"], ["3b4c2228_0"]]

# unique_correct_items=[]

# # iterate through the evaluation files and get the unique correct items
# for file in evaluation_files:
#     for item in stats_obj[file]['correct_items']:
#         for i in item:
#             if i not in unique_correct_items:
#                 unique_correct_items.append(i)
#                 # plot the riddle
#                 riddle = load_riddle_from_id(i.split("_")[-2].split("/")[-1])
#                 plot_task(riddle)
                
# # # set correct_items list equal to the "correct_items" column of stats_obj[file]
# # # iterate through correct items
# # for i in range(len(correct_items)):
# #     # get the correct items
# #     correct_items_for_epoch = correct_items[i]
# #     for correct_item in correct_items_for_epoch:
# #         if correct_item not in unique_correct_items:
# #             unique_correct_items.append(correct_item)

# # # iterate through the unique correct items
# # for unique_correct_item in unique_correct_items:
# #     riddle = load_riddle_from_id(unique_correct_item.split("_")[0])
# #     plot_task(riddle)


In [ ]:
# use this to continue training if the above cell fails or is stopped before completion
'''epoch_start=80
epoch_current=[]
epoch_current_count=epoch_start'''
#lr = 5e-4 # learning rate
""" epoch_start=2
epoch_current_count=epoch_start
for epochs in range(epoch_start+1, epoch_end, epoch_step):
    epoch_current_count+=epoch_step
    train_for_epochs_and_calc_totals(epoch_step, epoch_current_count, stats_obj)
    # need to add iteration through evaluation files
    save_data_and_stats(stats_obj) """

If you stop training early, you can use the below to run the model against the validation dataset and it will save the results to a file.

In [ ]:
# iterate through the evaluation_files
#for file in evaluation_files:
#    get_and_save_predictions(model, file)

===> Running inference...
load_and_process_dataset_from_file ./eval_data_none.csv
filter_file_for_max_tokens ./eval_data_none.csv
loading dataset from pandas dataframe


  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `LongT5ForConditionalGeneration.forward` and have been ignored: token_count_correct, id, number_to_replace_list, augmentation, prompt, token_count, replace_with_list, correct_answer. If token_count_correct, id, number_to_replace_list, augmentation, prompt, token_count, replace_with_list, correct_answer are not expected by `LongT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 104
  Batch size = 20
c:\ProgramData\Anaconda3\envs\ml\lib\site-packages\transformers\modeling_utils.py:667: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


  0%|          | 0/6 [00:00<?, ?it/s]

filter_file_for_max_tokens ./eval_data_none.csv


If you stop in the middle of a training set, you can save the Huggingface model and tokenizer with the code below.  This model does work across platforms.

In [ ]:
# clear gpu memory cache
#torch.cuda.empty_cache()

# Manually save model

If the training stops and you want to manually save the model, you can use the code below.

In [ ]:

#use this to save the huggingface model and tokenizer
model.save_pretrained(f"{experiment_folder}/model")
tokenizer.save_pretrained(f"{experiment_folder}/model")

Configuration saved in .//model\config.json


RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

# Features to add

Display all answers so you can see the difference between the correct answer and the predicted answers.  Maybe show correct answer and then all 4 of the parsed answer variations (see the results file for the 4 columns that are after correct answer).  Maybe show items that meet a minimum accuracy threshold to see what the issues are.

In [ ]:
#not using this function anymore. it is replaced by run_inference
def do_inference(model, input_text, return_error_status=True):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    error = False
    try:
        outputs = model.generate(input_ids, temperature=prediction_temp, num_beams=4, num_beam_groups=2, top_k=50, top_p=0.95, early_stopping=True, max_length=max_output_length, repetition_penalty=2.5)
    except IndexError:
        outputs = None
        error = True
        if return_error_status:
            return outputs, error
        else:
            return outputs
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # remove all characters that are not numbers, spaces, or commas
    decoded_output = re.sub(r"[^0-9, ]", "", decoded_output) + "."
    if return_error_status:
        return decoded_output, error
    else:
        return decoded_output

#do_inference(model, "train input1 10000 00020 output1 5 2 5 01 01000 00200. input2 01000 00200 output2 5 2 5 01 00100 02000. test tinput1 00100 00002 toutput1 ", return_error_status=True)